In [1]:
import pyaudio
import queue
import threading
import numpy as np
import math
import wave
import matplotlib.pyplot as plt
from scipy.io import wavfile


class MicArray:

    def __init__(self,device_index = None,rate = 16000,channels = 4,chunk_size = 1024):
        self.p = pyaudio.PyAudio()
        self.q = queue.Queue()
        self.thread_event = threading.Event()

        self.rate = rate
        self.channels = channels
        self.chunk_size = chunk_size
        
        if device_index == None:
            for i in range(self.p.get_device_count()):
                dev = self.p.get_device_info_by_index(i)
                name = dev['name'].encode('utf-8')
                print(i, name, dev['maxInputChannels'], dev['maxOutputChannels'])
                if dev['maxInputChannels'] == self.channels:
                    print('Use {}'.format(name))
                    device_index = i
                    break

                
        self.stream = self.p.open(
            input_device_index = device_index,
            start = False,
            format=pyaudio.paInt16,
            channels=self.channels,
            rate=self.rate,
            input=True,
            frames_per_buffer=self.chunk_size,
            stream_callback=self._callback
        )
               
    def _callback(self,input_data,frame_count,time_info,status_flag):
        self.q.put(input_data)
        return (None,pyaudio.paContinue)
    
    def start(self):
        self.q.queue.clear()
        self.stream.start_stream()
    
    def stop(self):
        self.thread_event.set()
        self.stream.stop_stream()
        self.q.put('')

    def read_mic_data(self):
        self.thread_event.clear()
        while not self.thread_event.is_set():
            frames = self.q.get()
            if not frames:
                break
            
            frames = np.frombuffer(frames,dtype = 'int16')
            yield frames

    def __enter__(self):
        self.start()
        return self

    def __exit__(self,exception_type,exception_value,traceback):
        if exception_value:
            return False
        self.stop()
        

def gccphat(sig,refsig,fs,max_tau,interp=1):
    
    n = sig.shape[0] + refsig.shape[0]

    SIG = np.fft.rfft(sig, n=n)
    REFSIG = np.fft.rfft(refsig, n=n)
    R = SIG * np.conj(REFSIG)

    cc = np.fft.irfft(R / np.abs(R), n=(interp * n))
    
    max_shift = np.minimum(int(interp * fs * max_tau), int(interp * n/2))
    
    cc = np.concatenate((cc[-max_shift:], cc[:max_shift+1]))
    
    shift = np.argmax(np.abs(cc)) - max_shift
    
    tau = shift / float(interp * fs)
    
    return tau,cc
    
    

In [2]:
def find_direction_gccphat(data_frame,fs,tau_mic_dist):
    tau,_ = gccphat(data_frame[0::4],data_frame[3::4],fs=fs,max_tau=tau_mic_dist,interp = 16)
    #print(tau)
    #print(tau_mic_dist)
    #print(tau/tau_mic_dist)
    #print(math.asin(tau/tau_mic_dist))
    theta = math.asin(tau/tau_mic_dist) * 180 / math.pi 
    
    return theta+90

def find_direction_gccphat2(data_frame1,data_frame2,fs,tau_mic_dist):
    tau,cc = gccphat(data_frame1,data_frame2,fs=fs,max_tau=tau_mic_dist)
    theta = math.asin(tau/tau_mic_dist) * 180 / math.pi      
    plt.plot(cc)
    
    return theta+90

def real_time_separate_mic():
    import signal
    import time
    
    distance = 0.0568
    max_time = distance/343.2
    Sampling_Rate = 16000
    
    is_quit = threading.Event()
    
    def signal_handler(sig, num):
        is_quit.set()
        print('Exited')
    
    signal.signal(signal.SIGINT, signal_handler)
    
    with MicArray(device_index=1,rate=Sampling_Rate,channels=1) as mic1, MicArray(device_index=2,rate=Sampling_Rate,channels=1) as mic2:
        for chunk1,chunk2 in zip(mic1.read_mic_data(),mic2.read_mic_data()):
            direction = find_direction_gccphat2(chunk1,chunk2,fs=Sampling_Rate,tau_mic_dist=max_time)
            print(int(direction))
            
            if is_quit.is_set():
                break

def mic_array_test():
    import signal
    import time
    
    distance = 0.0568
    max_time = distance/343.2
    
    is_quit = threading.Event()
    
    def signal_handler(sig, num):
        is_quit.set()
        print('Exited')
    
    signal.signal(signal.SIGINT, signal_handler)
    
    with MicArray(channels=4) as mic:
        for chunk in mic.read_mic_data():
            direction = find_direction_gccphat(chunk,fs=mic.rate,tau_mic_dist=max_time)
            print(int(direction))
            
            if is_quit.is_set():
                break


def from_wave_files():
    
    RATE = 16000
    distance = 0.0568
    max_time = distance/343.2
    
    wav = wave.open("data.wav")
    wav.setpos(0)
    sdata = wav.readframes(wav.getnframes())
    
    
    data = np.frombuffer(sdata, dtype=np.int16)
    direction = find_direction_gccphat(data,fs=RATE,tau_mic_dist=max_time)
    print(int(direction))
    

def from_two_files():
    RATE = 16000
    distance = 0.0568
    max_time = distance/343.2
    
    _, data1 = wavfile.read('./ch1.wav') 
    _, data2 = wavfile.read('./ch4.wav')
    
    
    direction = find_direction_gccphat2(data1,data2,fs=RATE,tau_mic_dist=max_time)
    print(int(direction))
    
def main():
    mic_array_test()
    #from_wave_files()
    #from_two_files()
    #real_time_separate_mic()
    
if __name__ == "__main__":
    main()
    

0 b'HDA Intel HDMI: 0 (hw:0,3)' 0 8
1 b'HDA Intel HDMI: 1 (hw:0,7)' 0 8
2 b'HDA Intel HDMI: 2 (hw:0,8)' 0 8
3 b'HDA Intel HDMI: 3 (hw:0,9)' 0 8
4 b'HDA Intel HDMI: 4 (hw:0,10)' 0 8
5 b'USB PnP Sound Device: Audio (hw:1,0)' 1 0
6 b'HDA Intel PCH: ALC3234 Analog (hw:2,0)' 2 2
7 b'USB Camera-B4.09.24.1: Audio (hw:3,0)' 4 0
Use b'USB Camera-B4.09.24.1: Audio (hw:3,0)'
105
92
91
90
92
91
143
148
150
91
88
85
91
91
92
96
99
105
92
92
98
91
94
103
95
94
95
100
96
99
110
100
96
98
96
95
94
94
100
96
153
91
148
94
100
99
119
94
70
95
91
95
31
40
26
26
66
66
92
85
7
115
172
73
92
92
100
58
44
7
7
7
131
91
38
91
96
40
85
88
92
91
122
121
88
96
91
102
102
98
102
19
103
100
107
103
109
102
105
107
106
92
105
105
103
91
94
91
96
91
87
98
91
94
99
94
95
98
103
99
115
126
105
105
100
103
99
95
96
92
92
96
98
99
99
99
106
102
107
102
99
102
99
94
105
106
105
103
95
94
99
99
90
94
50
106
95
92
103
99
109
102
95
99
99
102
106
105
100
96
102
96
100
110
99
96
94
91
145
91
105
92
Exited
95


Note: you may need to restart the kernel to use updated packages.


'C:\Users\Subash' is not recognized as an internal or external command,
operable program or batch file.
